In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**# This Data_Loader file **

In [0]:


import os
from os import listdir
from os.path import isfile, join
import numpy as np
from tensorflow.contrib import learn

# This Data_Loader file is copied online
class Data_Loader:
    def __init__(self, options):

        positive_data_file = options['dir_name']
        positive_examples = list(open(positive_data_file, "r").readlines())
        positive_examples = [s for s in positive_examples]


        max_document_length = max([len(x.split(",")) for x in positive_examples])
        vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
        self.item = np.array(list(vocab_processor.fit_transform(positive_examples)))
        self.item_dict = vocab_processor.vocabulary_._mapping


    def load_generator_data(self, sample_size):
        text = self.text
        mod_size = len(text) - len(text)%sample_size
        text = text[0:mod_size]
        text = text.reshape(-1, sample_size)
        return text, self.vocab_indexed


    def string_to_indices(self, sentence, vocab):
        indices = [ vocab[s] for s in sentence.split(',') ]
        return indices

    def inidices_to_string(self, sentence, vocab):
        id_ch = { vocab[ch] : ch for ch in vocab } 
        sent = []
        for c in sentence:
            if id_ch[c] == 'eol':
                break
            sent += id_ch[c]

        return "".join(sent)

   


**utils**

In [0]:
import numpy as np

def sample_top(a=[], top_k=10):
    idx = np.argsort(a)[::-1]
    idx = idx[:top_k]
    probs = a[idx]
    probs = probs / np.sum(probs)
    choice = np.random.choice(idx, p=probs)
    return choice

# fajie
def sample_top_k(a=[], top_k=10):
    idx = np.argsort(a)[::-1]
    idx = idx[:top_k]
    # probs = a[idx]
    # probs = probs / np.sum(probs)
    # choice = np.random.choice(idx, p=probs)
    return idx

print sample_top_k(np.array([0.02,0.01,0.01,0.16,0.8]),3)




[4 3 0]


**text_cnn_hv.py**

In [0]:
import tensorflow as tf
import numpy as np

'''
including verical and horizonal cnn
'''
class TextCNN_hv(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    "https://github.com/dennybritz/cnn-text-classification-tf"
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, loss_type, l2_reg_lambda):

        # Placeholders for input, output and dropout



        self.wholesession = tf.placeholder('int32',
                                         [None, None], name='wholesession')

        # a=self.t_sentence.get_shape()[1]*2


        source_sess = self.wholesession[:, 0:-1]
        target_sess = self.wholesession[:, -1:]

        new_sequence_length=sequence_length-1

        # source_embedding = tf.nn.embedding_lookup(self.wholesession,
        #                                           source_sess, name="source_embedding")
        # target_embedding=tf.nn.embedding_lookup(self.wholesession,
        #                                    target_sess, name="target_sess")


        # self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        # self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.input_x=source_sess
        self.input_y=target_sess

        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        self.loss_type = loss_type
        self.l2_reg_lambda = l2_reg_lambda


        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                # http://www.infoq.com/cn/articles/introduction-of-tensorflow-part4   how to use cnn
                # new shape after conv2d[?, new_sequence_length - filter_size + 1, 1, 1]
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                # new shape after max_pool[?, 1, 1, num_filters]
                # be carefyul, the  new_sequence_length has changed because of wholesession[:, 0:-1]
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, new_sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total]) #shape=[batch_size, 384]
        # design the veritcal cnn
        with tf.name_scope("conv-verical" ):
            filter_shape = [new_sequence_length, 1, 1, 1]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[1]), name="b")
            conv = tf.nn.conv2d(
                self.embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
        self.vcnn_flat= tf.reshape(h, [-1, embedding_size])
        self.final=tf.concat([self.h_pool_flat,self.vcnn_flat],1) #shape=[batch_size, 384+100]




        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.final, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total+embedding_size, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            # self.predictions = tf.argmax(self.scores, 1, name="predictions")
            # losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.input_y = tf.reshape(self.input_y, [-1])
            self.loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y, logits=self.scores)
            self.loss = tf.reduce_mean(self.loss + l2_reg_lambda * l2_loss)

            self.probs_flat = tf.nn.softmax(self.scores)
            self.arg_max_prediction = tf.argmax(self.probs_flat, 1)


        # Calculate mean cross-entropy loss
        # with tf.name_scope("loss"):
        #     if self.loss_type == 'square_loss':
        #         if self.l2_reg_lambda > 0:
        #             self.loss = tf.nn.l2_loss(
        #                 tf.subtract(self.input_y, self.scores)) +  l2_reg_lambda * l2_loss  # regulizer
        #         else:
        #             self.loss = tf.nn.l2_loss(tf.subtract(self.input_y, self.scores))






        # Accuracy
        # with tf.name_scope("accuracy"):
        #     correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
        #     self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")


**参数类**

In [0]:
class Args(object):
    """docstring for Hotel"""
    def __init__(self,
                 learning_rate = 0.001,
                 batch_size=10, 
                 sample_every=2000,
                 summary_every=50,
                 save_model_every = 1500,
                 sample_size=300,
                 top_k = 5,
                 max_epochs = 50,
                 beta1 = 0.5,
                 resume_model = None,
                 text_dir ='/content/gdrive/My Drive/test.csv' ,
                 data_dir = '/content/gdrive/My Drive/',
                 seed='f78c95a8-9256-4757-9a9f-213df5c6854e,1151b040-8022-4965-96d2-8a4605ce456c',
                 sample_percentage = 0.2,
                 filter_sizes = '[2,3,4]',
                 num_filters = 100,#128
                 loss_type = 'square_loss',#'Specify a loss type (square_loss or log_loss).'
                 l2_reg_lambda = 0,
                 allow_soft_placement = True,
                 log_device_placement = False,
                 dropout_keep_prob =0.5
                 
                ):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.sample_every = sample_every
        self.summary_every = summary_every
        self.save_model_every = save_model_every
        self.sample_size = sample_size
        self.top_k = top_k
        self.max_epochs = max_epochs
        self.beta1 =beta1
        self.resume_model = resume_model
        self.text_dir=text_dir
        self.data_dir=data_dir
        self.seed=seed
        self.sample_percentage=sample_percentage
        self.filter_sizes=filter_sizes
        self.num_filters=num_filters
        self.loss_type=loss_type
        self.l2_reg_lambda=l2_reg_lambda
        self.allow_soft_placement=allow_soft_placement
        self.log_device_placement=log_device_placement
        self.dropout_keep_prob=dropout_keep_prob


**caser**

In [0]:
import tensorflow as tf
import numpy as np
import argparse
import shutil
import time
import os
import sys
import math


'''
reimplementation of
Personalized Top-N Sequential Recommendation via
Convolutional Sequence Embedding
screen print has been changed a bit so that to print the output not that ofen
'''
def main():
   
    args = Args()



    dl = Data_Loader({'model_type': 'generator', 'dir_name': args.text_dir})
    # text_samples=16390600  vocab=947255  session100

    all_samples = dl.item
    items = dl.item_dict


    # Randomly shuffle data
    np.random.seed(10)
   
    shuffle_indices = np.random.permutation(np.arange(len(all_samples)))
    text_samples = all_samples[shuffle_indices]



    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(args.sample_percentage * float(len(text_samples)))
    x_train, x_dev = text_samples[:dev_sample_index], text_samples[dev_sample_index:]

    #create subsession only for training
    subseqtrain = []
    for i in range(len(x_train)):
        #print x_train[i]
        seq=x_train[i]
        lenseq=len(seq)
        #session lens=100 shortest subsession=5 realvalue+95 0
        for j in range(lenseq-4):
            subseqend=seq[:len(seq)- j]
            subseqbeg=[0]*j
            subseq= np.append(subseqbeg,subseqend)
            #beginseq=padzero+subseq
            #newsubseq=pad+subseq
            subseqtrain.append(subseq)
    x_train=np.array(subseqtrain) #list to ndarray
    del subseqtrain
    # Randomly shuffle data
    np.random.seed(10)
    shuffle_train = np.random.permutation(np.arange(len(x_train)))
    x_train = x_train[shuffle_train]
    print "generating subsessions is done!"
    print "shape", x_train.shape[0]
    print "dataset",args.text_dir
    model_options = {
        'vocab_size': len(items),
        'residual_channels': 100,
    }


    cnn = TextCNN_hv(
        sequence_length=x_train.shape[1],
        num_classes=len(items),
        vocab_size=len(items),
        embedding_size= model_options['residual_channels'],
        filter_sizes=eval(args.filter_sizes),
        num_filters=args.num_filters,
        loss_type=args.loss_type,
        l2_reg_lambda=args.l2_reg_lambda)

    print "embedding_size", model_options['residual_channels']
    session_conf = tf.ConfigProto(
        # allow to distribute device automatically if your assigned device is not found
        allow_soft_placement=args.allow_soft_placement,
        # whether print or not
        log_device_placement=args.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        sess.run(tf.global_variables_initializer())





    step = 1
    for epoch in range(args.max_epochs):
        batch_no = 0
        batch_size = args.batch_size
        while (batch_no + 1) * batch_size < x_train.shape[0]:

            start = time.clock()

            text_batch = x_train[batch_no * batch_size: (batch_no + 1) * batch_size, :]

            _, loss, prediction = sess.run(
                [train_op, cnn.loss,
                 cnn.arg_max_prediction],
                feed_dict={
                    cnn.wholesession: text_batch,
                    cnn.dropout_keep_prob: args.dropout_keep_prob
                })
            end = time.clock()
            if step % args.sample_every == 0:
                print "-------------------------------------------------------train1"
                print "LOSS: {}\tEPOCH: {}\tBATCH_NO: {}\t STEP:{}\t total_batches:{}".format(
                    loss, epoch, batch_no, step, x_train.shape[0] / args.batch_size)
                print "TIME FOR BATCH", end - start
                print "TIME FOR EPOCH (mins)", (end - start) * (x_train.shape[0] / args.batch_size) / 60.0

            # print "-------------------------------------------------------train2"
            # loss = sess.run(
            #     [generator_model.loss_test],
            #     feed_dict={
            #         generator_model.seed_sentence: text_batch
            #     })
            # print "LOSS: {}\tEPOCH: {}\tBATCH_NO: {}\t STEP:{}\t total_batches:{}".format(
            #     loss, epoch, batch_no, step, x_train.shape[0] / args.batch_size)

            # print "-------------------------------------------------------test1"
            # if (batch_no + 1) * batch_size < x_dev.shape[0]:
            #     text_batch = x_dev[(batch_no) * batch_size: (batch_no + 1) * batch_size, :]
            # loss = sess.run(
            #     [generator_model.loss],
            #     feed_dict={
            #         generator_model.t_sentence: text_batch
            #     })
            # print "LOSS: {}\tEPOCH: {}\tBATCH_NO: {}\t STEP:{}\t total_batches:{}".format(
            #     loss, epoch, batch_no, step, x_dev.shape[0] / args.batch_size)
            if step % args.sample_every == 0:
                print "-------------------------------------------------------test1"
                if (batch_no + 1) * batch_size < x_dev.shape[0]:
                    text_batch = x_dev[(batch_no) * batch_size: (batch_no + 1) * batch_size, :]
                loss = sess.run(
                    [cnn.loss],
                    feed_dict={
                        cnn.wholesession: text_batch,
                        cnn.dropout_keep_prob: 1.0
                    })
                print "LOSS: {}\tEPOCH: {}\tBATCH_NO: {}\t STEP:{}\t total_batches:{}".format(
                    loss, epoch, batch_no, step, x_dev.shape[0] / args.batch_size)



            batch_no += 1

            if step % args.sample_every == 0:
                print "********************************************************accuracy"
                batch_no_test = 0
                batch_size_test = batch_size*2
                curr_preds_5 = []
                rec_preds_5 = []  # 1
                ndcg_preds_5 = []  # 1
                curr_preds_20 = []
                rec_preds_20 = []  # 1
                ndcg_preds_20 = []  # 1
                while (batch_no_test + 1) * batch_size_test < x_dev.shape[0]:
                    # do not need to evaluate all, only after several 10 sample_every, then output final results
                    if(step / (args.sample_every)<10):
                        if (batch_no_test > 2):
                            break
                    else:
                        if (batch_no_test > 500):
                            break
                    text_batch = x_dev[batch_no_test * batch_size_test: (batch_no_test + 1) * batch_size_test, :]
                    [probs] = sess.run(
                        [cnn.probs_flat],
                        feed_dict={
                            cnn.wholesession: text_batch,
                            cnn.dropout_keep_prob: 1.0
                        })
                    for bi in range(probs.shape[0]):
                        pred_words_5 = sample_top_k(probs[bi], top_k=args.top_k)  # top_k=5
                        pred_words_20 = sample_top_k(probs[bi], top_k=args.top_k + 15)

                        true_word = text_batch[bi][-1]
                        predictmap_5 = {ch: i for i, ch in enumerate(pred_words_5)}
                        pred_words_20 = {ch: i for i, ch in enumerate(pred_words_20)}

                        rank_5 = predictmap_5.get(true_word)
                        rank_20 = pred_words_20.get(true_word)
                        if rank_5 == None:
                            curr_preds_5.append(0.0)
                            rec_preds_5.append(0.0)  # 2
                            ndcg_preds_5.append(0.0)  # 2
                        else:
                            MRR_5 = 1.0 / (rank_5 + 1)
                            Rec_5 = 1.0  # 3
                            ndcg_5 = 1.0 / math.log(rank_5 + 2, 2)  # 3
                            curr_preds_5.append(MRR_5)
                            rec_preds_5.append(Rec_5)  # 4
                            ndcg_preds_5.append(ndcg_5)  # 4
                        if rank_20 == None:
                            curr_preds_20.append(0.0)
                            rec_preds_20.append(0.0)  # 2
                            ndcg_preds_20.append(0.0)  # 2
                        else:
                            MRR_20 = 1.0 / (rank_20 + 1)
                            Rec_20 = 1.0  # 3
                            ndcg_20 = 1.0 / math.log(rank_20 + 2, 2)  # 3
                            curr_preds_20.append(MRR_20)
                            rec_preds_20.append(Rec_20)  # 4
                            ndcg_preds_20.append(ndcg_20)  # 4

                    batch_no_test += 1
                    print "BATCH_NO: {}".format(batch_no_test)
                    print "Accuracy mrr_5:", sum(curr_preds_5) / float(len(curr_preds_5))  # 5
                    print "Accuracy mrr_20:", sum(curr_preds_20) / float(len(curr_preds_20))  # 5
                    print "Accuracy hit_5:", sum(rec_preds_5) / float(len(rec_preds_5))  # 5
                    print "Accuracy hit_20:", sum(rec_preds_20) / float(len(rec_preds_20))  # 5
                    print "Accuracy ndcg_5:", sum(ndcg_preds_5) / float(len(ndcg_preds_5))  # 5
                    print "Accuracy ndcg_20:", sum(ndcg_preds_20) / float(len(ndcg_preds_20))  #
                    # print "curr_preds",curr_preds

            step += 1





if __name__ == '__main__':
    main()


Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
generating subsessions is done!
shape 1444
dataset /content/gdrive/My Drive/test.csv
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
embedding_size 100
Instructions for updating:
Use tf.cast instead.
-------------------------------------------------------train1
LOSS: 0.0860433652997	EPOCH: 13	BATCH_NO: 127	 STEP:2000	 total_batches:144
TIME FOR BATCH 0.009735
TIME FOR EPOCH (mins) 0.023364
-------------------------------------------------------test1
LOSS: [0.017618757]	EPOCH: 13	BATCH_NO: 127	 STEP:2000	 total_batches:36
********************************************************accuracy
BATCH_NO: 1
Accuracy mrr_5: 0.1
Accuracy mrr_20: 0.10263